In [1]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import numpy as np
import json
import requests
import config

In [2]:
# Store csv path ad name
ship_csv = ("clean.csv")
postcode_csv = ("data/australian_postcodes.csv")
sa3_csv = ("data/SA3_2021_AUST.csv")

# Read in csv file
shipments_df = pd.read_csv(ship_csv, index_col=0)
postcode_df = pd.read_csv(postcode_csv)
sa3_df = pd.read_csv(sa3_csv)

### Shipment Data

In [3]:
# get names of indexes for which column cmlPostCode has value "H7E 1A1"
postcode_index_names = shipments_df[ shipments_df['cmlPostCode'] == "H7E 1A1" ].index

# get names of indexes for which column smpPlantID has value "P00"
index_names = shipments_df[ shipments_df['smpPlantID'] == "P00" ].index
  
# drop these row indexes from dataFrame

shipments_df.drop(postcode_index_names, inplace = True)
shipments_df.drop(index_names, inplace = True)

In [4]:
# Reset index

shipments_df = shipments_df.reset_index(drop=True)

In [5]:
# Convert datatype to numeric

shipments_df[["postcode"]] = shipments_df[["cmlPostCode"]].apply(pd.to_numeric)

# Convert postcode datatype to int
shipments_df[["postcode"]] = shipments_df[["postcode"]].astype(np.int64)

# Drop old postcode column
shipments_df = shipments_df.drop('cmlPostCode', inplace=False, axis=1)

In [6]:
# Convert columns to date type

shipments_df['smpShipDate'] = pd.to_datetime(shipments_df['smpShipDate'].str.strip(), format='%d/%m/%Y %H:%M:%S %p')
shipments_df['smlCreatedDate'] = pd.to_datetime(shipments_df['smlCreatedDate'].str.strip(), format='%d/%m/%Y %H:%M:%S %p')

In [7]:
# Add new column to hold period

shipments_df['ShipPeriod'] = shipments_df['smpShipDate'].dt.to_period('M').astype(str)
shipments_df['ShipYear'] = shipments_df['smpShipDate'].dt.to_period('Y').astype(str)
shipments_df['ShipMonth'] = shipments_df['smpShipDate'].dt.month_name()

In [8]:
# Print Head

shipments_df.head(5)

,smpPlantID,smlCreatedDate,smlShipmentID,smlShipmentLineID,smlPartID,smlDescription,smlPartRevisionID,smpCustomerOrganizationID,smpShipDate,smpShipOrganizationID,...,cmlOrganizationID,cmlName,cmlAddressLine1,cmlCity,cmlState,quantityShipped,postcode,ShipPeriod,ShipYear,ShipMonth
0,P01,2021-07-23 04:41:16,129241,1,105689,Cardboard Carton Dc/Rsc 1190mm x 195mm x 700mm...,NaN,FROAUS,2021-07-26 12:00:00,FROAUS,...,FROAUS,Frontline Australasia Pty Ltd,"Door 3, 55 Letcon Drive",Bangholme,VIC,23.0,3175,2021-07,2021,July
1,P01,2021-07-23 04:41:16,129241,2,105689,Cardboard Carton Dc/Rsc 1190mm x 195mm x 700mm...,NaN,FROAUS,2021-07-26 12:00:00,FROAUS,...,FROAUS,Frontline Australasia Pty Ltd,"Door 3, 55 Letcon Drive",Bangholme,VIC,3.0,3175,2021-07,2021,July
2,P09,2021-07-26 06:35:23,129247,1,305834,WESTRO Pallet 1165mm x 1165mm x 138mm,P09,WESOCE,2021-07-26 06:35:17,WESOCE,...,WESOCE,Westrock Oceania Pty Ltd,"Gate 1, Beaumont Avenue",North Richmond,NSW,20.0,2754,2021-07,2021,July
3,P09,2021-07-26 06:36:49,129248,1,306036,FIRES Pallet 570mm x 570mm,P09,FIRPTY,2021-07-26 06:36:44,FIRPTY,...,FIRPTY,Firesense Pty Ltd,18-20 Brookhollow Avenue,Baulkham Hills,NSW,32.0,2153,2021-07,2021,July
4,P09,2021-07-26 06:36:49,129248,2,305915,Std Heavy Duty Second Hand Pallets,HEAVY DUTY,FIRPTY,2021-07-26 06:36:44,FIRPTY,...,FIRPTY,Firesense Pty Ltd,18-20 Brookhollow Avenue,Baulkham Hills,NSW,20.0,2153,2021-07,2021,July


### Cleaning Postcode Data

In [9]:
# Drop unnecessary columns
filtered_postcode_df = postcode_df[["postcode","lat","long","sa3","sa3name"]]

# Drop NaN values in sa3 column
filtered_postcode_df = filtered_postcode_df.dropna(subset=['sa3'])

# Check to see how many uniqu postcodes are in the df
len(filtered_postcode_df["postcode"].unique())


3073

In [10]:
# Ceate new df with duplicated postcodes removed
unique_postcode_df = filtered_postcode_df.drop_duplicates(['postcode'], keep='first')

# # Set sa3 as the index
unique_postcode_df["sa3"] = unique_postcode_df["sa3"].astype(np.int64)

C:\Users\joshu\anaconda3\envs\PythonData2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [11]:
# Confirm no. of unique rows are still in the df
len(unique_postcode_df)

3073

In [12]:
# Print Head
unique_postcode_df.head(5)

,postcode,lat,long,sa3,sa3name
2,800,-12.458684,130.836680,70101,Darwin City
4,801,-12.458684,130.836680,70101,Darwin City
5,804,-12.428017,130.873315,70102,Darwin Suburbs
6,810,-12.381806,130.866242,70102,Darwin Suburbs
22,811,-12.381806,130.866242,70102,Darwin Suburbs


### Joining postcode df and shipment df

In [13]:
# Combine the data into a single dataset
shipment_merge_df = pd.merge(shipments_df, unique_postcode_df, on="postcode")

In [14]:
shipment_merge_df.head(5)

,smpPlantID,smlCreatedDate,smlShipmentID,smlShipmentLineID,smlPartID,smlDescription,smlPartRevisionID,smpCustomerOrganizationID,smpShipDate,smpShipOrganizationID,...,cmlState,quantityShipped,postcode,ShipPeriod,ShipYear,ShipMonth,lat,long,sa3,sa3name
0,P01,2021-07-23 04:41:16,129241,1,105689,Cardboard Carton Dc/Rsc 1190mm x 195mm x 700mm...,NaN,FROAUS,2021-07-26 12:00:00,FROAUS,...,VIC,23.0,3175,2021-07,2021,July,-38.016114,145.208504,21204,Dandenong
1,P01,2021-07-23 04:41:16,129241,2,105689,Cardboard Carton Dc/Rsc 1190mm x 195mm x 700mm...,NaN,FROAUS,2021-07-26 12:00:00,FROAUS,...,VIC,3.0,3175,2021-07,2021,July,-38.016114,145.208504,21204,Dandenong
2,P02,2021-07-01 09:34:29,131919,1,300085,2 Way Pallet 1165mm x 1165mm x 124mm P1,NaN,ROBBOS,2021-07-01 09:33:35,ROBBOS,...,VIC,100.0,3175,2021-07,2021,July,-38.016114,145.208504,21204,Dandenong
3,P02,2021-07-01 09:34:30,131919,2,300068,2 Way Pallet 1100mm x 1100mm x 144mm Export P4,NaN,ROBBOS,2021-07-01 09:33:35,ROBBOS,...,VIC,126.0,3175,2021-07,2021,July,-38.016114,145.208504,21204,Dandenong
4,P02,2021-06-30 04:16:31,131985,1,300949,Bulk Pack 1910mm x 950mm x 595mm 203,NaN,FROAUS,2021-07-01 04:15:00,FROAUS,...,VIC,8.0,3175,2021-07,2021,July,-38.016114,145.208504,21204,Dandenong


### Cleaning SA3 Data

In [15]:
# Drop unnecessary columns
filtered_sa3_df = sa3_df[['SA3_CODE_2021',"SA3_NAME_2021","ASGS_LOCI_URI_2021"]]

#Rename Columns
filtered_sa3_df = filtered_sa3_df.rename(columns = {'SA3_CODE_2021':'sa3','SA3_NAME_2021':'sa3name', 'ASGS_LOCI_URI_2021':'location_URI'
                              }, inplace = False)

# Set sa3 as the index
filtered_sa3_df = filtered_sa3_df.set_index("sa3", inplace = False)

# Drop Index ZZZZZZ
clean_sa3_df = filtered_sa3_df.drop('ZZZZZ')

# Print Head
clean_sa3_df.head(5)

,sa3name,location_URI
sa3,,
10102,Queanbeyan,http://linked.data.gov.au/dataset/asgsed3/SA3/...
10103,Snowy Mountains,http://linked.data.gov.au/dataset/asgsed3/SA3/...
10104,South Coast,http://linked.data.gov.au/dataset/asgsed3/SA3/...
10105,Goulburn - Mulwaree,http://linked.data.gov.au/dataset/asgsed3/SA3/...
10106,Young - Yass,http://linked.data.gov.au/dataset/asgsed3/SA3/...


In [16]:
# API - requesting coordinates for the sa3 ID. Data to be used to plot on maps

# Add new column to the DataFrame
clean_sa3_df["coorinates"] = ""
clean_sa3_df["type"] = ""

# Iterate through the all rows
for index, row in clean_sa3_df.iterrows():
    
    id = index


    # setup query url
    query_url = f"https://asgs.linked.fsdf.org.au/dataset/asgsed3/collections/SA3/items/{id}?_profile=oai&_mediatype=application/geo+json"


    # make request and convert to json
    sa3_json = requests.get(query_url).json()
    
 # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        
        # Adding collected data to sa3 dataframe
        clean_sa3_df.loc[index, "coorinates"] = sa3_json["geometry"]["coordinates"][0]
        clean_sa3_df.loc[index, "type"] = sa3_json["geometry"]["type"]
        
        print(f"id. {index} found")
                  
        # Add except 
    except (KeyError, IndexError):
        print(f"SA3 data not found. Skipping id. {index}....")
                  
        # Remove row if no data found
        clean_sa3_df.drop(index, inplace=True)

id. 10102 found
id. 10103 found
id. 10104 found
id. 10105 found
id. 10106 found
id. 10201 found
id. 10202 found
id. 10301 found
id. 10302 found
id. 10303 found
id. 10304 found
id. 10401 found
id. 10402 found
id. 10501 found
id. 10502 found
id. 10503 found
id. 10601 found
id. 10602 found
id. 10603 found
id. 10604 found
id. 10701 found
id. 10702 found
id. 10703 found
id. 10704 found
id. 10801 found
id. 10802 found
id. 10803 found
id. 10804 found
id. 10805 found
id. 10901 found
id. 10902 found
id. 10903 found
id. 11001 found
id. 11002 found
id. 11003 found
id. 11004 found
id. 11101 found
id. 11102 found
id. 11103 found
id. 11201 found
id. 11202 found
id. 11203 found
id. 11301 found
id. 11302 found
id. 11303 found
id. 11401 found
id. 11402 found
id. 11501 found
id. 11502 found
id. 11503 found
id. 11504 found
id. 11601 found
id. 11602 found
id. 11603 found
id. 11701 found
id. 11702 found
id. 11703 found
id. 11801 found
id. 11802 found
id. 11901 found
id. 11902 found
id. 11903 found
id. 1190

### Load Data

In [17]:
#Connecting to local database using config for all personal data
rds_connection_string = f'{config.protocol}://{config.username}:{config.password}@{config.host}:{config.port}/{config.database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

In [18]:
#Check to see if queries.sql code has been run in PGAdmin
insp.get_table_names()

['sa3_table', 'shipment_table']

In [19]:
#Import sa3_table data frame into SQL
clean_sa3_df.to_sql(name='sa3_table', con=engine, if_exists='append', index=True)

In [20]:
#Checking if sa3_table data has been loaded into the database
pd.read_sql_query('select * from sa3_table', con=engine).head()

,sa3,sa3name,coorinates,location_URI,type
0,10102,Queanbeyan,"[[150.03913680728206, -35.33689791038227], [15...",http://linked.data.gov.au/dataset/asgsed3/SA3/...,Polygon
1,10103,Snowy Mountains,"[[149.09592193071893, -35.81759754163424], [14...",http://linked.data.gov.au/dataset/asgsed3/SA3/...,Polygon
2,10104,South Coast,"[[[149.9459842247479, -36.94921522282815], [14...",http://linked.data.gov.au/dataset/asgsed3/SA3/...,MultiPolygon
3,10105,Goulburn - Mulwaree,"[[149.29848461668564, -34.73877515165846], [14...",http://linked.data.gov.au/dataset/asgsed3/SA3/...,Polygon
4,10106,Young - Yass,"[[148.00653987761302, -34.34748471148191], [14...",http://linked.data.gov.au/dataset/asgsed3/SA3/...,Polygon


In [21]:
#Import australian_postcodes data frame into SQL
shipment_merge_df.to_sql(name='shipment_table', con=engine, if_exists='append', index=False)

In [22]:
#Checking if australian_postcodes data has been loaded into the database
pd.read_sql_query('select * from shipment_table', con=engine).head()

,id,smpPlantID,smlCreatedDate,smlShipmentID,smlShipmentLineID,smlPartID,smlDescription,smlPartRevisionID,smpCustomerOrganizationID,smpShipDate,...,cmlState,quantityShipped,ShipPeriod,ShipYear,ShipMonth,postcode,lat,long,sa3,sa3name
0,1,P01,2021-07-23,129241,1,105689,Cardboard Carton Dc/Rsc 1190mm x 195mm x 700mm...,None,FROAUS,2021-07-26,...,VIC,23.0,2021-07,2021,July,3175,-38.016114,145.208504,21204,Dandenong
1,2,P01,2021-07-23,129241,2,105689,Cardboard Carton Dc/Rsc 1190mm x 195mm x 700mm...,None,FROAUS,2021-07-26,...,VIC,3.0,2021-07,2021,July,3175,-38.016114,145.208504,21204,Dandenong
2,3,P02,2021-07-01,131919,1,300085,2 Way Pallet 1165mm x 1165mm x 124mm P1,None,ROBBOS,2021-07-01,...,VIC,100.0,2021-07,2021,July,3175,-38.016114,145.208504,21204,Dandenong
3,4,P02,2021-07-01,131919,2,300068,2 Way Pallet 1100mm x 1100mm x 144mm Export P4,None,ROBBOS,2021-07-01,...,VIC,126.0,2021-07,2021,July,3175,-38.016114,145.208504,21204,Dandenong
4,5,P02,2021-06-30,131985,1,300949,Bulk Pack 1910mm x 950mm x 595mm 203,None,FROAUS,2021-07-01,...,VIC,8.0,2021-07,2021,July,3175,-38.016114,145.208504,21204,Dandenong
